In [5]:
import tkinter as tk
from PIL import Image, ImageTk
from tkinter import messagebox
from plyer import notification
import serial
import threading
from datetime import datetime

def send_command(command):
    ser.write(command.encode())

def on_command_click(command):
    send_command(command)
    notification_text = ser.readline().decode().strip()

    # Menentukan judul notifikasi berdasarkan jenis perintah
    if command in ('A', 'B'):
        title = 'Printer'
    elif command in ('C', 'D'):
        title = 'Card'
    elif command in ('E', 'F'):
        title = 'LED Cash Out 1'
    elif command in ('G', 'H'):
        title = 'LED Cash Out 2'
    else:
        title = 'Notification'

    # Menampilkan notifikasi
    notification.notify(
        title=f'Control {title} UNDIP X UNNES',
        message=notification_text,
        app_name='Microcontroller Notifier'
    )
    
    # Menambahkan respon ke label
    label_response.config(text=f"Respon: {notification_text}")


def read_serial():
    while not exit_event.is_set():
        try:
            if ser.is_open and ser.in_waiting:
                notification_text = ser.readline().decode().strip()
                notification.notify(
                    title='Notifikasi Mikrokontroler',
                    message=notification_text,
                    app_name='Microcontroller Notifier'
                )
                update_response(notification_text)
        except serial.SerialException as e:
            print(f"Serial Exception: {e}")
            break 

def on_exit():
    if ser.is_open:
        ser.close()
    root.destroy()

def update_response(new_response):
    timestamp = datetime.now().strftime("[%H:%M:%S] ")
    current_responses = response_text.get("1.0", tk.END).strip()
    new_responses = f"{current_responses}\n{timestamp}{new_response}"
    response_text.config(state=tk.NORMAL)
    response_text.delete("1.0", tk.END)
    response_text.insert(tk.END, new_responses)
    response_text.yview(tk.END)  # Menggeser tampilan ke bagian bawah
    response_text.config(state=tk.DISABLED)

def start_serial_reading():
    serial_thread = threading.Thread(target=read_serial)
    serial_thread.start()

def clear_response():
    response_text.config(state=tk.NORMAL)
    response_text.delete("1.0", tk.END)
    response_text.config(state=tk.DISABLED)

# Ganti dengan port UART yang sesuai (COMx pada Windows atau /dev/ttyUSBx pada Linux)
serial_port = "COM3"
baud_rate = 9600

try:
    ser = serial.Serial(serial_port, baud_rate, timeout=1)

    root = tk.Tk()
    root.title("Control LED and Limit Switch UNDIP X UNNES")

    button_frame = tk.Frame(root)
    button_frame.pack(pady=10)

    #Menambahkan tombol untuk Clear Respons
    clear_button = tk.Button(root, text="Clear", command=clear_response)
    clear_button.pack(pady=10)

    # Menambahkan tombol untuk Printer
    printer_frame = tk.Frame(button_frame)
    printer_frame.pack(side=tk.LEFT, padx=5)
    button_blink = tk.Button(printer_frame, text="Printer ON", command=lambda: on_command_click('A'))
    button_blink.pack(side=tk.TOP, pady=5)
    button_blink = tk.Button(printer_frame, text="Printer OFF", command=lambda: on_command_click('B'))
    button_blink.pack(side=tk.TOP, pady=5)

    # Menambahkan tombol untuk CRT
    crt_frame = tk.Frame(button_frame)
    crt_frame.pack(side=tk.LEFT, padx=5)
    button_on = tk.Button(crt_frame, text="Card In", command=lambda: on_command_click('C'))
    button_on.pack(side=tk.TOP, pady=5)
    button_on = tk.Button(crt_frame, text="Card Out", command=lambda: on_command_click('D'))
    button_on.pack(side=tk.TOP, pady=5)

    # Menambahkan tombol untuk LED Cash Out 1
    cash_out_1_frame = tk.Frame(button_frame)
    cash_out_1_frame.pack(side=tk.LEFT, padx=5)
    button_on = tk.Button(cash_out_1_frame, text="Cash Out ON", command=lambda: on_command_click('E'))
    button_on.pack(side=tk.TOP, pady=5)
    button_on = tk.Button(cash_out_1_frame, text="Cash Out OFF", command=lambda: on_command_click('F'))
    button_on.pack(side=tk.TOP, pady=5)

    # Menambahkan tombol untuk LED Cash Out 2
    cash_out_2_frame = tk.Frame(button_frame)
    cash_out_2_frame.pack(side=tk.LEFT, padx=5)
    button_on = tk.Button(cash_out_2_frame, text="Cash Out 2 ON", command=lambda: on_command_click('G'))
    button_on.pack(side=tk.TOP, pady=5)
    button_on = tk.Button(cash_out_2_frame, text="Cash Out 2 OFF", command=lambda: on_command_click('H'))
    button_on.pack(side=tk.TOP, pady=5)

    exit_button = tk.Button(root, text="Keluar", command=on_exit)
    exit_button.pack(pady=10)

    #label respon LED
    label_response = tk.Label(root, text="Respon: ")
    label_response.pack(pady=10)

    #label respon Limitswitch
    response_text = tk.Text(root, height=10, width=50, state=tk.DISABLED)
    response_text.pack(pady=10)

    root.protocol("WM_DELETE_WINDOW", on_exit)

    exit_event = threading.Event() 
    start_serial_reading() 

    root.mainloop()

except serial.SerialException as e:
    messagebox.showerror("Error", f"Serial Error: {e}")
finally:
    if ser.is_open:
       ser.close()
    exit_event.set()

Serial Exception: ClearCommError failed (OSError(9, 'The handle is invalid.', None, 6))
